# Problem 1:

## 0. Given code

In [2]:
import os
import pandas as pd
import numpy as np

# Initialize
def get_movie_data():
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames)
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames)
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames)

    return users, ratings, movies

users, ratings, movies = get_movie_data()

tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

data = pd.merge(movies, ratings)
data = pd.merge(data, users)

D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


## 1. Print out the top rated movie

In [3]:
print('The highest rated movie is', data.sort_values(by=['rating'], ascending = False).iloc[0]['title'])

The highest rated movie is Toy Story (1995)


## 2. Find top n date night movies

### a. Cleanup data

In [4]:
data = data[data.age > 15]    # Exclude reviews by people too young to date
data = data.drop_duplicates(['short_title', 'user_id'])  # Why would you review a movie twice

### b. Calculate gender appeal

In [5]:
# Pivot around user gender
gen_data = data.pivot(index = 'short_title', columns = 'user_id', values= ['gender'])
gen_data = gen_data.apply(lambda x: pd.Series(x[~x.isnull()].values), axis=1)  # Collapse columns

# Count male and female reviewers
gen_data["f_count"] = gen_data.apply(lambda row: sum(row[0:-1]=='F') ,axis=1)
gen_data["m_count"] = gen_data.apply(lambda row: sum(row[0:-1]=='M') ,axis=1)

# Get difference between male and female reviewers
gen_data["m/f"] = gen_data.apply(lambda row: abs(row[-1] - row[-2]), axis = 1)

# Sort by title
gen_data.sort_index(axis = 0, ascending = True, inplace = True)

gen_data.head()

,0,1,2,3,4,5,6,7,8,9,...,3329,3330,3331,3332,3333,3334,3335,f_count,m_count,m/f
short_title,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck",M,F,M,M,F,F,F,M,M,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,21,5
'Night Mother,M,F,M,F,M,M,M,M,M,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,33,2
'Til There Was You,F,F,F,M,F,F,F,F,M,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,14,22
"'burbs, The",F,M,F,F,M,M,M,F,M,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92,209,117
...And Justice for All,M,F,F,M,F,F,M,M,M,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,163,128


### c. Calculate average rating

In [7]:
# Pivot around ratings
rat_data = data.pivot(index = 'short_title', columns = 'user_id', values= ['rating'])
rat_data = rat_data.apply(lambda x: pd.Series(x[~x.isnull()].values), axis=1)  # Collapse columns

#Get and round average rating
rat_data['avg_rating'] = rat_data.mean(numeric_only=True, axis=1)
rat_data['avg_rating'] = rat_data['avg_rating'].apply(np.int64)

# Sort by title
rat_data.sort_index(axis = 0, ascending = True, inplace = True)

rat_data.head()

,0,1,2,3,4,5,6,7,8,9,...,3327,3328,3329,3330,3331,3332,3333,3334,3335,avg_rating
short_title,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck",2.0,5.0,4.0,1.0,3.0,5.0,4.0,3.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
'Night Mother,5.0,3.0,3.0,5.0,5.0,3.0,1.0,3.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
'Til There Was You,2.0,1.0,4.0,3.0,4.0,3.0,3.0,4.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
"'burbs, The",4.0,4.0,2.0,4.0,2.0,5.0,3.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...And Justice for All,2.0,3.0,5.0,3.0,4.0,4.0,3.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


### d. Analysis

In [8]:
# Construct new table from calculated values
drop_data = pd.DataFrame(rat_data['avg_rating'])
drop_data['f_count'] = gen_data['f_count']
drop_data['m_count'] = gen_data['m_count']
drop_data['m/f'] = gen_data['m/f']

# Exclude unpopular movies
drop_data = drop_data[gen_data.f_count > 100]
drop_data = drop_data[gen_data.m_count > 100]

# Group by rating, select the highest rated movies
grouped = drop_data.groupby(['avg_rating'])
drop_data = grouped.get_group(4)

# Sort by male/female popularity
drop_data = drop_data.sort_values(by = 'm/f', ascending = True)
drop_data.head()

D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,avg_rating,f_count,m_count,m/f
short_title,,,,
Roman Holiday,4,207,202,5
Sense and Sensibility,4,407,412,5
Top Hat,4,114,132,18
Rebecca,4,167,208,41
My Fair Lady,4,283,330,47


### e. Format & print

In [9]:
movie_list = ''
for i in range(10):
    if i == 9:
        movie_list += ' and '
    movie_list += drop_data.index[i]
    if i < 9:
        movie_list += ', '
        
print('The best date night movies are ', movie_list, '.')

The best date night movies are  Roman Holiday, Sense and Sensibility, Top Hat, Rebecca, My Fair Lady, Raise the Red Lantern, Shall We Dance? (Shall We Dansu?), Eat Drink Man Woman, Room with a View, A,  and Cat on a Hot Tin Roof .


# Problem 2:
## 0. Given code

In [287]:
from IPython.core.display import HTML
import pandas as pd
import numpy as np

ref = HTML(filename='../data/titanic.html')

t_file = pd.ExcelFile('../data/titanic.xls')
t_df = t_file.parse("titanic", header=None)
t_df = t_df.rename(columns=t_df.iloc[0]).drop(t_df.index[0])

## 1. Survival vs sex

In [288]:
grouped = t_df.groupby(['survived', 'sex'])
sex = ['female', 'male']

for gender in sex:
    try:
        alive = len(grouped.get_group((1, gender)))
        dead = len(grouped.get_group((0, gender)))
        print('\nSurvival rates for ', gender, 's: ', alive/(dead+alive))
    except:
        continue


Survival rates for  female s:  0.7274678111587983

Survival rates for  male s:  0.19098457888493475


## 2. Survival vs sex vs class

In [289]:
# Sex vs class vs survivorship
grouped = t_df.groupby(['survived', 'sex', 'pclass'])
pclass = [1, 2, 3]

for gender in sex:
    print('\nSurvival rates for ', gender, 's: ')
    for rank in pclass:
        try:
            alive = len(grouped.get_group((1, gender, rank)))
            dead = len(grouped.get_group((0, gender, rank)))
            print('\tClass ', rank, ': ', alive/(dead+alive))
        except:
            continue


Survival rates for  female s: 
	Class  1 :  0.9652777777777778
	Class  2 :  0.8867924528301887
	Class  3 :  0.49074074074074076

Survival rates for  male s: 
	Class  1 :  0.3407821229050279
	Class  2 :  0.14619883040935672
	Class  3 :  0.15212981744421908


## 3. Survival vs sex vs class vs age

### Bucketing function

In [290]:
def buckets (age):
    if age < 14: return 0
    elif age < 21: return 1
    elif age < 65: return 2
    else: return 3
    
t_df['age'] = t_df['age'].apply(lambda pt: buckets(pt))

### Calculations

In [291]:
grouped = t_df.groupby(['survived', 'sex', 'pclass', 'age'])
age = [0, 1, 2, 3]

for gender in sex:
    print('\nSurvival rates for ', gender, 's: ')
    for rank in pclass:
        print('\tClass ', rank, ': ')
        for old in age:
            try:
                alive = len(grouped.get_group((1, gender, rank, old)))
                dead = len(grouped.get_group((0, gender, rank, old)))
                print('\t\tAge group ', old, ': ', alive/(dead+alive))
            except:
                continue


Survival rates for  female s: 
	Class  1 : 
		Age group  2 :  0.9655172413793104
	Class  2 : 
		Age group  1 :  0.9230769230769231
		Age group  2 :  0.868421052631579
		Age group  3 :  0.6666666666666666
	Class  3 : 
		Age group  0 :  0.4838709677419355
		Age group  1 :  0.5428571428571428
		Age group  2 :  0.4418604651162791
		Age group  3 :  0.53125

Survival rates for  male s: 
	Class  1 : 
		Age group  1 :  0.2
		Age group  2 :  0.34328358208955223
		Age group  3 :  0.2571428571428571
	Class  2 : 
		Age group  1 :  0.11764705882352941
		Age group  2 :  0.078125
		Age group  3 :  0.13333333333333333
	Class  3 : 
		Age group  0 :  0.32432432432432434
		Age group  1 :  0.12307692307692308
		Age group  2 :  0.1598360655737705
		Age group  3 :  0.10884353741496598
